###**Q1**.  Use [Lucidchart](https://sjsu.edu/it/services/applications/lucidchart.php) to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
import sqlite3
import pandas as pd

# Create SQLite connection
conn = sqlite3.connect('store.db')
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE product (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price REAL
);
""")
cursor.execute("""
CREATE TABLE customer (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT NOT NULL
);
""")
cursor.execute("""
CREATE TABLE purchase_order (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    date TEXT NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
);
""")
cursor.execute("""
CREATE TABLE order_item (
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    PRIMARY KEY (order_id, product_id),
    FOREIGN KEY (order_id) REFERENCES purchase_order(order_id),
    FOREIGN KEY (product_id) REFERENCES product(product_id)
);
""")
conn.commit()

In [2]:
def load_csv_to_table(csv_url, table_name):
    df = pd.read_csv(csv_url)
    df.to_sql(table_name, conn, if_exists='append', index=False)

load_csv_to_table('https://raw.githubusercontent.com/csbfx/cs133/main/product.csv', 'product')
load_csv_to_table('https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv', 'customer')
load_csv_to_table('https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv', 'purchase_order')
load_csv_to_table('https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv', 'order_item')
conn.commit()

In [3]:
# Verify tables
for table in ['product', 'customer', 'purchase_order', 'order_item']:
    print(f"Data in {table}:")
    print(pd.read_sql_query(f"SELECT * FROM {table};", conn))
    print("\n")


Data in product:
   product_id     name  price
0           0  bicycle  400.0
1           1   helmet   45.0
2           2   gloves   23.0
3           3    chain   48.0


Data in customer:
   customer_id               name            email
0            0          Wendy Lee    wlee@bike.com
1            1        Jason Brown     jb@speed.com
2            2       Harry Potter  hp@hogwarts.edu
3            3  Godric Gryffindor  gg@hogwards.edu


Data in purchase_order:
   order_id  customer_id        date
0         0            0  2020-10-19
1         1            0  2020-10-20
2         2            1  2020-10-20


Data in order_item:
   order_id  product_id  quantity
0         0           0         1
1         0           1         1
2         1           2         2
3         1           3         1




In [5]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .
# Verify data loaded into each table
tables = ['product', 'customer', 'purchase_order', 'order_item']

for table in tables:
    print(f"Data in {table} table:")
    data = pd.read_sql_query(f"SELECT * FROM {table};", conn)
    print(data)
    print("\n")



Data in product table:
   product_id     name  price
0           0  bicycle  400.0
1           1   helmet   45.0
2           2   gloves   23.0
3           3    chain   48.0


Data in customer table:
   customer_id               name            email
0            0          Wendy Lee    wlee@bike.com
1            1        Jason Brown     jb@speed.com
2            2       Harry Potter  hp@hogwarts.edu
3            3  Godric Gryffindor  gg@hogwards.edu


Data in purchase_order table:
   order_id  customer_id        date
0         0            0  2020-10-19
1         1            0  2020-10-20
2         2            1  2020-10-20


Data in order_item table:
   order_id  product_id  quantity
0         0           0         1
1         0           1         1
2         1           2         2
3         1           3         1




In [11]:
conn.commit()

In [13]:
# Query to find purchase date, product, and quantity for a specific customer
cursor.execute(
    """
    SELECT
        customer.name AS customer_name,
        purchase_order.date,
        product.name AS product_name,
        order_item.quantity
    FROM
        customer
    JOIN
        purchase_order ON customer.customer_id = purchase_order.customer_id
    JOIN
        order_item ON order_item.order_id = purchase_order.order_id
    JOIN
        product ON product.product_id = order_item.product_id
    WHERE
        customer.name = 'Wendy Lee'
    """
)

# Fetch and display results
results = cursor.fetchall()
print(results)


[('Wendy Lee', '2020-10-19', 'helmet', 1), ('Wendy Lee', '2020-10-20', 'gloves', 2), ('Wendy Lee', '2020-10-20', 'chain', 1)]


In [6]:
# Additional tasks
# Appending a new row to one of the tables
# Deleting row(s) with "XXX"

# Append a new row to the `product` table
new_product = (101, 'New Product', 19.99)
cursor.execute("INSERT INTO product (product_id, name, price) VALUES (?, ?, ?);", new_product)
conn.commit()

# Verify the new row
print("Updated `product` table after appending:")
print(pd.read_sql_query("SELECT * FROM product;", conn))


Updated `product` table after appending:
   product_id         name   price
0           0      bicycle  400.00
1           1       helmet   45.00
2           2       gloves   23.00
3           3        chain   48.00
4         101  New Product   19.99


In [8]:
# Delete rows with "XXX" in the `name` field
cursor.execute("DELETE FROM product WHERE name = 'bicycle';")
conn.commit()

# Verify deletion
print("Updated `product` table after deleting rows with 'XXX':")
print(pd.read_sql_query("SELECT * FROM product;", conn))


Updated `product` table after deleting rows with 'XXX':
   product_id         name  price
0           1       helmet  45.00
1           2       gloves  23.00
2           3        chain  48.00
3         101  New Product  19.99
